In [ ]:
###
# This code is used for pre-augmentation (Augmentation 3 in the paper) 
###

In [ ]:
import os, glob, shutil, tifffile, json, pathlib
import numpy as np
from PIL import Image, ImageFilter, ImageDraw, ImageOps, ImageEnhance, ImageMath  
from joblib import Parallel, delayed
from tqdm import tqdm_notebook as tqdm
from concurrent.futures import ThreadPoolExecutor

seed = 0

%matplotlib inline

In [ ]:
orig_d = '/PATH/TO/YOUR/IMAGES'
img_d = os.path.join(orig_d, 'img')
lab_d = os.path.join(orig_d, 'lab')

imgs = sorted(glob.glob(img_d + '/*.png'))
labs = sorted(glob.glob(lab_d + '/*.txt'))

print(len(imgs))
print(len(labs))

In [ ]:
temp = []
for i, j in zip(imgs, labs):
    temp.append(os.path.basename(os.path.splitext(i)[0]) == os.path.basename(os.path.splitext(j)[0]))
print(all(temp))

In [ ]:
def check_yolo_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            class_id = int(line.split()[0])
            if class_id == 2 or class_id == 3:
                return True
    return False



In [ ]:
temp = []
for i in labs:
    if check_yolo_file(i):
        print(i)
        temp.append(i)

In [ ]:
def process_file(i):
    if check_yolo_file(i):
        img_fna = os.path.dirname(os.path.dirname(i)) + '/img/' + os.path.basename(os.path.splitext(i)[0]) + '.png'
        if not os.path.exists(img_fna):
            print(f"The file {img_fna} does not exist.")
            return
        
        img = Image.open(img_fna)
        
        img_inverted = ImageOps.invert(img)

        enhancer = ImageEnhance.Color(img)
        img_enhance_half = enhancer.enhance(0.5)
        img_enhance_zero = enhancer.enhance(0.0)

        h, s, v = img.convert("HSV").split()
        _h = ImageMath.eval("(h + 64 + 128) % 255", h=h).convert("L")
        img_hsv = Image.merge("HSV", (_h, s, v)).convert("RGB")

        autocont = ImageOps.autocontrast(img)
            
        poster3 = ImageOps.posterize(img, 3)
        poster4 = ImageOps.posterize(img, 4)
        
        quantize12 = img.quantize(12) 
        quantize6 = img.quantize(6) 
      
        solar200 = ImageOps.solarize(img, 200)
        solar160 = ImageOps.solarize(img, 160)
        
        gray = img.convert("L")    
        gray_3 = Image.merge("RGB", (gray, gray, gray))        
        bin180 = gray_3.point(lambda x: 0 if x < 180 else x)
        bin200 = gray_3.point(lambda x: 0 if x < 200 else x)
        sepia = Image.merge("RGB",(gray.point(lambda x: x * 240 / 255),
                                   gray.point(lambda x: x * 200 / 255),
                                   gray.point(lambda x: x * 145 / 255)))

        light12 = img.point(lambda x: x * 1.2) 
        dark05 = img.point(lambda x: x * 0.5)  
        
        detail = img.filter(ImageFilter.DETAIL)
        sharpen = img.filter(ImageFilter.SHARPEN)
        edge = img.filter(ImageFilter.EDGE_ENHANCE)
        smooth = img.filter(ImageFilter.SMOOTH)
        smooth_more = img.filter(ImageFilter.SMOOTH_MORE)
        mode = img.filter(ImageFilter.ModeFilter(5))
        
        # Save augmented images
        img_inverted.save(os.path.splitext(img_fna)[0] + '_inverted.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_inverted.txt')
        
        img_enhance_half.save(os.path.splitext(img_fna)[0] + '_enhance_half.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_enhance_half.txt')
        
        img_enhance_zero.save(os.path.splitext(img_fna)[0] + '_enhance_zero.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_enhance_zero.txt')
        
        img_hsv.save(os.path.splitext(img_fna)[0] + '_hsv.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_hsv.txt')
        
        autocont.save(os.path.splitext(img_fna)[0] + '_autocont.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_autocont.txt')
        
        poster3.save(os.path.splitext(img_fna)[0] + '_poster3.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_poster3.txt')  
        
        poster4.save(os.path.splitext(img_fna)[0] + '_poster4.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_poster4.txt') 
 
        quantize12.save(os.path.splitext(img_fna)[0] + '_quantize12.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_quantize12.txt') 
 
        quantize6.save(os.path.splitext(img_fna)[0] + '_quantize6.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_quantize6.txt') 
        
        solar200.save(os.path.splitext(img_fna)[0] + '_solar200.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_solar200.txt')
        
        solar160.save(os.path.splitext(img_fna)[0] + '_solar160.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_solar160.txt')

        bin180.save(os.path.splitext(img_fna)[0] + '_bin180.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_bin180.txt')

        bin200.save(os.path.splitext(img_fna)[0] + '_bin200.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_bin200.txt')
        
        sepia.save(os.path.splitext(img_fna)[0] + '_sepia.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_sepia.txt')
        
        light12.save(os.path.splitext(img_fna)[0] + '_light12.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_light12.txt')
        
        dark05.save(os.path.splitext(img_fna)[0] + '_dark05.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_dark05.txt')
        
        detail.save(os.path.splitext(img_fna)[0] + '_detail.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_detail.txt')
        
        sharpen.save(os.path.splitext(img_fna)[0] + '_sharpen.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_sharpen.txt')
        
        edge.save(os.path.splitext(img_fna)[0] + '_edge.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_edge.txt')
        
        smooth.save(os.path.splitext(img_fna)[0] + '_smooth.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_smooth.txt')
        
        smooth_more.save(os.path.splitext(img_fna)[0] + '_smooth_more.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_smooth_more.txt')
        
        mode.save(os.path.splitext(img_fna)[0] + '_mode.png')
        shutil.copyfile(i, os.path.splitext(i)[0] + '_mode.txt')

In [ ]:
with ThreadPoolExecutor(max_workers=32) as executor:
    list(tqdm(executor.map(process_file, labs), total=len(labs)))

In [ ]:
os.rename(orig_d, orig_d +'_23AUG_enhanced')